In [89]:
import requests
from dotenv import load_dotenv
import os
import numpy as np
import pandas as pd
load_dotenv()
API_KEY = os.getenv('API_KEY')

In [79]:
def get_all_sports():
    sports_url = f'https://api.the-odds-api.com/v4/sports/?apiKey={API_KEY}'

    sports_response = requests.get(sports_url)
    sports_data = sports_response.json()

    sports = []
    for sport in sports_data:
        sports.append(sport['key'])

    return sports

def get_sport_teams_odds(sport):
    sport_team_odds_url = f'https://api.the-odds-api.com/v4/sports/{sport}/odds/?apiKey={API_KEY}&regions=us&markets=h2h,spreads&oddsFormat=american'
    response = requests.get(sport_team_odds_url)
    data = response.json()
    
    if len(data) <= 1:
        return None
    
    sport_team_odds = []
    for team in data:
        
        sports_teams = f"{team['bookmakers'][0]['markets'][0]['outcomes'][0]['name']} vs {team['bookmakers'][0]['markets'][0]['outcomes'][1]['name']}"
        
        books = []
        odds1 = []
        odds2 = []
        for odds in team['bookmakers']:
            books.append(odds['title'])
            odds1.append(odds['markets'][0]['outcomes'][0]['price'])
            odds2.append(odds['markets'][0]['outcomes'][1]['price']) 
        
        data = {
                'Books': books,
                'Odds1': odds1,
                'Odds2': odds2
            }
        team_odds = (sports_teams, pd.DataFrame(data))
        sport_team_odds.append(team_odds)
    return sport_team_odds

def calc_arb_opp(sport_odds):
    
    for game_index in range(len(sport_odds)):
        max_odds1 = sport_odds[game_index][1]['Odds1'].max()
        max_odds2 = sport_odds[game_index][1]['Odds2'].max()
        
        implied_prob = find_probs(max_odds1, max_odds2) 
        if implied_prob < 100:
            max_odds1_book = sport_odds.loc[sport_odds[game_index][1]['Odds1'].idxmax(), 'Books']
            max_odds2_book = sport_odds.loc[sport_odds[game_index][1]['Odds2'].idxmax(), 'Books']

            print(f'Arbitrage Opportunity Avalible: {sport_odds[game_index][0]}, {max_odds1_book}:{max_odds1} | {max_odds2_book}:{max_odds2}')
        return implied_prob 
        
def find_probs(b1, b2):
    
    if b1 < 0:
        p1 = b1 / (b1 - 100)
    else:
        p1 = 100 / (b1 + 100)
        
    if b2 < 0:
        p2 = b2 / (b2 - 100)
    else:
        p2 = 100 / (b2 + 100)
        
    return p1 + p2

In [ ]:
sports = get_all_sports()

In [88]:
sports_team_data = get_sport_teams_odds(sports[0])
print(calc_arb_opp(sports_team_data))